In [71]:
%pip install prophet 

   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   -------------------- ------------------- 6.8/13.3 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------  13.1/13.3 MB 34.3 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 26.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [235]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from prophet import Prophet 

In [236]:
import pandas as pd
import numpy as np
import plotly as plt
import joblib
from prophet.diagnostics import cross_validation,performance_metrics

In [ ]:
#get the data ready

In [237]:
rev=pd.read_csv(r"C:\Users\silve\Desktop\Data Science\Projects\Trade Franchises\Meta Data\Raw data\biweekly_revenue_data.csv")
rev.head(1)

,date,store_id,customer_id,revenue
0,2023-01-01,7,1,4313.56


In [238]:
rev.fillna(0,inplace=True)


In [40]:
ave=st["revenue"].mean()
ave

778.6891666666667

In [240]:

st["revenue"]=st["revenue"].replace(0,ave)

In [241]:
rev=rev.drop(rev[rev["store_id"].isin([1,2,3,4,5,6,7,8,9])].index) # removing most of the stores for testing purpose 

In [242]:
rev=rev.drop(columns="store_id", axis=1)

In [244]:
rev.rename(columns={"customer_id":"Store_id"}, inplace=True)

In [ ]:
#use the model 

In [245]:
sa=Prophet(changepoint_prior_scale=0.5)

In [246]:
temp=st
temp=temp[["date","revenue"]]
temp=temp.rename(columns={"date":"ds","revenue":"y"})

In [247]:
sa.add_seasonality(name="biweekly", period=28, fourier_order=3)

In [248]:
sa.fit(temp)

23:27:06 - cmdstanpy - INFO - Chain [1] start processing
23:27:07 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
# predicting the next biweekly revenue

In [249]:
future=sa.make_future_dataframe(periods=1, freq='14D')


In [251]:
forcast=sa.predict(future)
forcast.tail(1)


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,biweekly,biweekly_lower,biweekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
24,2023-12-03,60.860638,706.315328,801.656327,56.890968,64.827302,692.376115,692.376115,692.376115,692.376115,692.376115,692.376115,0.0,0.0,0.0,753.236753


In [126]:
df_cv=cross_validation(sa, initial='280 days', period='14 days', horizon='42 days') # 280 days from jan-nov. 3 period 14*3 

  0%|          | 0/1 [00:00<?, ?it/s]

21:06:55 - cmdstanpy - INFO - Chain [1] start processing
21:06:55 - cmdstanpy - INFO - Chain [1] done processing


In [127]:
df_p=performance_metrics(df_cv)

In [128]:
print(df_p.head())

  horizon          mse       rmse        mae      mape     mdape     smape  \
0 14 days  3862.818384  62.151576  62.151576  0.076280  0.076280  0.079305   
1 28 days   696.199280  26.385588  26.385588  0.035220  0.035220  0.034610   
2 42 days  1066.817290  32.662169  32.662169  0.045965  0.045965  0.044932   

   coverage  
0       1.0  
1       1.0  
2       1.0  


In [ ]:
# save the model in folder

In [129]:
joblib.dump(sa,r"C:\Users\silve\Desktop\Data Science\Projects\Trade Franchises\Notebooks\Models\prophet_model.pkl")

['C:\\Users\\silve\\Desktop\\Data Science\\Projects\\Trade Franchises\\Notebooks\\Models\\prophet_model.pkl']

In [ ]:
# test the saved model

In [130]:
load=joblib.load(r"C:\Users\silve\Desktop\Data Science\Projects\Trade Franchises\Notebooks\Models\prophet_model.pkl")

In [252]:
futures_dates=load.make_future_dataframe(periods=1, freq='14 d') # createing new dates that will get predicted. 

In [253]:
futures_number=load.predict(futures_dates)

In [254]:
futures_number.tail(1)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,biweekly,biweekly_lower,biweekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
24,2023-12-03,56.291768,696.049043,824.339,56.291683,56.291838,706.301238,706.301238,706.301238,706.301238,706.301238,706.301238,0.0,0.0,0.0,762.593006


In [ ]:
#saving the next week prediction to excel  

In [234]:
forcast.to_excel(r"C:\Users\silve\Desktop\Data Science\Projects\Trade Franchises\Notebooks\Models\next_rev_pred.xlsx", index=False)